In [2]:
import numpy as np
from bayspec.model.local import *
from bayspec.model.astro import *
from bayspec import DataUnit, Data, Infer, Plot

In [3]:
le = DataUnit(
    src='./LE/le2.src', 
    bkg='./LE/le2.bkg', 
    rmf='./LE/le2.rmf', 
    arf='./LE/le2.arf', 
    notc=[0.5, 4], 
    stat='cstat', 
    grpg={'min_evt': 5})

me = DataUnit(
    src='./ME/me2.src', 
    bkg='./ME/me2.bkg', 
    rsp='./ME/me2.rsp', 
    notc=[8, 900], 
    stat='pgstat', 
    grpg={'min_evt': 50})

data = Data([('LE', le), ('ME', me)])

print(data)

╒════════╤════════════╤═════════════╤═════════════════╤════════╕
│  Name  │  Noticing  │  Statistic  │    Grouping     │  Time  │
╞════════╪════════════╪═════════════╪═════════════════╪════════╡
│   LE   │ [[0.5, 4]] │    cstat    │ {'min_evt': 5}  │  None  │
├────────┼────────────┼─────────────┼─────────────────┼────────┤
│   ME   │ [[8, 900]] │   pgstat    │ {'min_evt': 50} │  None  │
╘════════╧════════════╧═════════════╧═════════════════╧════════╛



In [4]:
tbabs_ins = AS_TbAbs()

cpl_ins = cpl()

model = tbabs_ins * cpl_ins

print(model)

(AS_TbAbs*cpl) [add]
AS_TbAbs: Photometric absorption (Tbabs implementation), f(E) = exp(- NH * sigma(E)) contributed by Dominique Eckert
cpl: cutoff power law model
╒════════╤═════════════╤═════════════╤═════════╕
│  cfg#  │  Component  │  Parameter  │  Value  │
╞════════╪═════════════╪═════════════╪═════════╡
│   1    │  AS_TbAbs   │  redshift   │    0    │
├────────┼─────────────┼─────────────┼─────────┤
│   2    │     cpl     │  redshift   │    0    │
╘════════╧═════════════╧═════════════╧═════════╛
╒════════╤═════════════╤═════════════╤═════════╤══════════════════╤══════════╕
│  par#  │  Component  │  Parameter  │  Value  │      Prior       │  Frozen  │
╞════════╪═════════════╪═════════════╪═════════╪══════════════════╪══════════╡
│   1    │  AS_TbAbs   │     NH      │    1    │ unif(0.0001, 20) │  False   │
├────────┼─────────────┼─────────────┼─────────┼──────────────────┼──────────┤
│   2    │     cpl     │  $\alpha$   │   -1    │   unif(-8, 4)    │  False   │
├────────┼───────

In [5]:
infer = Infer([(data, model)])
print(infer)

╒════════╤════════════════╤═════════════╤═════════════╤═════════╕
│  cfg#  │   Expression   │  Component  │  Parameter  │  Value  │
╞════════╪════════════════╪═════════════╪═════════════╪═════════╡
│   1    │ (AS_TbAbs*cpl) │  AS_TbAbs   │  redshift   │    0    │
├────────┼────────────────┼─────────────┼─────────────┼─────────┤
│   2    │ (AS_TbAbs*cpl) │     cpl     │  redshift   │    0    │
╘════════╧════════════════╧═════════════╧═════════════╧═════════╛
╒════════╤════════════════╤═════════════╤═════════════╤═════════╤══════════════════╕
│  par#  │   Expression   │  Component  │  Parameter  │  Value  │      Prior       │
╞════════╪════════════════╪═════════════╪═════════════╪═════════╪══════════════════╡
│   1*   │ (AS_TbAbs*cpl) │  AS_TbAbs   │     NH      │    1    │ unif(0.0001, 20) │
├────────┼────────────────┼─────────────┼─────────────┼─────────┼──────────────────┤
│   2*   │ (AS_TbAbs*cpl) │     cpl     │  $\alpha$   │   -1    │   unif(-8, 4)    │
├────────┼────────────────┼─

In [6]:
post = infer.emcee(nstep=1000, resume=True, savepath='./intermediate')
print(post)

100%|██████████| 1000/1000 [00:25<00:00, 38.86it/s]


╒════════╤════════════════╤═════════════╤═════════════╤════════╤══════════╤════════╤══════════════════╕
│  par#  │   Expression   │  Component  │  Parameter  │  Mean  │  Median  │  Best  │    1sigma CI     │
╞════════╪════════════════╪═════════════╪═════════════╪════════╪══════════╪════════╪══════════════════╡
│   1    │ (AS_TbAbs*cpl) │  AS_TbAbs   │     NH      │ 1.154  │  1.143   │ 1.098  │  [0.949, 1.359]  │
├────────┼────────────────┼─────────────┼─────────────┼────────┼──────────┼────────┼──────────────────┤
│   2    │ (AS_TbAbs*cpl) │     cpl     │  $\alpha$   │ -1.675 │  -1.684  │ -1.659 │ [-1.755, -1.595] │
├────────┼────────────────┼─────────────┼─────────────┼────────┼──────────┼────────┼──────────────────┤
│   3    │ (AS_TbAbs*cpl) │     cpl     │ log$E_{c}$  │  2.95  │  2.909   │ 2.761  │  [2.418, 3.570]  │
├────────┼────────────────┼─────────────┼─────────────┼────────┼──────────┼────────┼──────────────────┤
│   4    │ (AS_TbAbs*cpl) │     cpl     │   log$A$    │ -3.064 │

In [7]:
fig = Plot.infer(post, style='CE', show=True)
fig.write_html('intermediate/ctsspec.html')

In [8]:
fig = Plot.infer(post, style='NE', show=True)
fig.write_html('intermediate/phtspec.html')

In [9]:
fig = Plot.post_corner(post, show=True)
fig.write_html('intermediate/corner.html')

In [10]:
earr = np.logspace(np.log10(0.5), 4, 100)

modelplot = Plot.model(ploter='plotly', style='vFv', CI=True)
fig = modelplot.add_model(model, E=earr, show=False)
fig = modelplot.add_model(cpl_ins, E=earr, show=True)
fig.write_html('intermediate/model.html')